In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\End_to_End_ML_projects\\Text-Summarizer-End-to-End-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\LENOVO\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-08-24 18:23:00,211: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-24 18:23:00,218: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-24 18:23:00,221: INFO: common: created directory at: artifacts]
[2023-08-24 18:23:00,224: INFO: common: created directory at: artifacts/data_transformation]
[2023-08-24 18:23:01,532: INFO: filelock: Lock 1867616083872 acquired on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\0a6130bfb0bfa44a3a376fe225b327e7ba8fa889.lock]


[2023-08-24 18:23:02,266: INFO: filelock: Lock 1867616083872 released on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\0a6130bfb0bfa44a3a376fe225b327e7ba8fa889.lock]



c:\Users\LENOVO\anaconda3\envs\textS\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[2023-08-24 18:23:02,586: INFO: filelock: Lock 1867616083536 acquired on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\2c1a911e577525af99c26c1634c473667e1e7ae2.lock]


[2023-08-24 18:23:02,893: INFO: filelock: Lock 1867616083536 released on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\2c1a911e577525af99c26c1634c473667e1e7ae2.lock]


[2023-08-24 18:23:03,236: INFO: filelock: Lock 1867616458688 acquired on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\5dba785a4a5acd022e91efdf53a9966f91d6e7d8.lock]


[2023-08-24 18:23:05,371: INFO: filelock: Lock 1867616458688 released on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\5dba785a4a5acd022e91efdf53a9966f91d6e7d8.lock]
[2023-08-24 18:23:06,221: INFO: filelock: Lock 1867616458928 acquired on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\6dc4d430ddbd24171268d73da061ce9f0b092911.lock]


[2023-08-24 18:23:06,575: INFO: filelock: Lock 1867616458928 released on C:\Users\LENOVO/.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs\6dc4d430ddbd24171268d73da061ce9f0b092911.lock]



Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\LENOVO\anaconda3\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 27094.22 examples/s]
